<a href="https://colab.research.google.com/github/saberforce/Portfolio-Component-Risk-Return-Landscape/blob/main/coingeckoinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycoingecko


In [87]:
import pandas as pd
import requests

In [86]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()


In [ ]:
list_coin={}
wholelist=['bitcoin', 'ethereum', 'wmatic','chainlink', 'uniswap','the-sandbox','decentraland','the-graph','aave', 'wrapped-usdc']

# Allocated % Inside The Portfolio In Ordered Sequence
wholelistportion=[0.2,0.2,0.1,0.1,0.05,0.05,0.05,0.05,0.05,0.15]

# For Each Coin Get A List Of Prices Over The Last 365 Days
for coin in wholelist:
  gg=cg.get_coin_market_chart_by_id(id=coin,vs_currency='usd',days='364')
  listOfPrices=[]
  num_prices=range(len(gg['prices']))
  for k in num_prices:

    coinprice=(gg['prices'][k][1])
    listOfPrices.append(coinprice)


 #Prep data - deficient data count for some coins
  list_coin[f'{coin}']=pd.Series(listOfPrices)

# Getting A List of Returns Derived From Daily Prices For Each Coin, The Expected 
# Mean Of Return For Each Coin, The Standard Deviation For Each Coin
# Deriving The Portfolio Mean And Portfolio Standard Deviation

df_gg=pd.DataFrame()
n=0
portfoliomean=0
portfoliostd=0

returnlist = [x+"_returns" for x in wholelist]
meanlist=[]
stdlist=[]


for k,v in list_coin.items():
  df_gg[f'{k}']=v
  df_gg[returnlist[n]]=df_gg[f'{k}'].pct_change(1)
  
  g=df_gg[returnlist[n]].mean()*100
  h=df_gg[returnlist[n]].std()*100

  portfoliomean=portfoliomean+(wholelistportion[n]*g)

  portfoliostd=portfoliostd+(wholelistportion[n]*h)

  meanlist.append(g)
  stdlist.append(h)
  n=n+1

wholelist.append('portfolio')
meanlist.append(portfoliomean)
stdlist.append(portfoliostd)

# Finalizing The Finished Dataframe Tabulation For 9 Coins And 1 Portfolio Item

listOfFinals = list(zip(wholelist, meanlist, stdlist))

df_final = pd.DataFrame(listOfFinals, columns = ['name', 'mean','std'])
df_final





In [126]:
import plotly.express as px



In [157]:
fig = px.scatter(df_final, x='std', y='mean', range_y=[-0.5,3],
                 hover_name='name', 
                 color='name', 
                 size="mean",
                 text="name",
                 title="Risk Return Landscape",



                 )


fig.show()